# Magic Tricks: Demystifying IPython Magics

> Jupyter notebooks support a feature called "magic commands" (aka "IPython magics") that use a special syntax for calling utility functions. This talk explains how magics work and how to write custom magics that make security analysis and research tasks more efficient. The live demo is a proof-of-concept magic command that integrates the Azure CLI with pandas for convenient evidence collection and analysis in Jupyter notebooks.

**Ryan Marcotte Cobb (ryan@detect.dev)**

Principal Security Researcher, Secureworks

---

## whoami

- 10 years DFIR and research at Secureworks
- Contributor to FOSS projects: `msticpy`, `stratus-red-team`, `ROADtools`
- Developer of Jupyter-based tools for security automation

---

## Overview

- Introduction to IPython magics
- Write our own custom IPython magics
- Examples of using IPython magics for Infosec
    - `ROADtools`
    - DFIR report automation
    - Azure CLI

_Follow along on binderhub!_

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/secureworks/infosec-jupyterthon-2022-ipython-magics/HEAD?labpath=preso.ipynb)

---

## Introduction to IPython Magics

- [Tutorial on Magic Functions](https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#magic-functions)
- [Magic command system](https://ipython.readthedocs.io/en/stable/interactive/reference.html#magic)

IPython provides a programmatic interface for tools that were not designed for Python, but are generally Unix-like (usually commandline).

These conventions unify Python, the browser, and native tools.

IPython uses a **special syntax** to tell the interpreter" that some code should be evaluated differently than normal Python: `?`, `!`, `%`, and `%%`.

---

### `!` Magic Syntax

`!` (the "bang" character) is the IPython symbol for a shell command -- wrapping `bash`, `cmd`, and/or `pwsh`.

In [ ]:
!whoami

You can use these special IPython syntaxes wherever Python statements are valid.

Values can be are assigned to variables based on the `stdout` output from the system shell.

In [ ]:
directory_listing = !dir
directory_listing


---

### `?` Magic Syntax

The `?` (question mark) character is the IPython equivalent of a `man page`, but for displaying Python docstrings.

In [ ]:
import pandas as pd

df = pd.DataFrame()
df?


---

### `%` and `%%` Magic Syntax

But I am here to discuss user-defined **magic** commands, which use the `%` and `%%` operators.

### Line Magic

The single `%` operator indicates a **line magic**.

The remainder of the current line is evaluated by the magic, and the return value can be assigned to variables in the namespace wherever Python statements can be made.

You can list all available magics using the `%magic` command.

In [ ]:
%magic

### Cell Magic

The `%%` syntax indicates a **cell magic**.

The contents of the notebook cell are passed to the magic function.

In [ ]:
%%html
<button>click me</button>

Typically, cell magics parse the first line of the cell as its arguments.

In [ ]:
%%writefile foo.txt
Lorem ipsum
Whatever

The same magic can also behave as either or cell magic.


---

## Writing a Custom IPython Magic

| Magic Type | Function Signature |
| ---------- | ------------------ |
| Line       | `def my_line_magic(line: str)` |
| Cell       | `def my_cell_magic(line: str, cell: str)` |
| Either     | `def my_combo_magic(line: str, cell: str = None)` |

### Example 1 - Define Custom Magics via Decorators

In [ ]:
# Ex. 1
# https://ipython.readthedocs.io/en/stable/config/custommagics.html

from typing import Tuple, Union

from IPython.core.magic import (
    register_line_magic,
    register_cell_magic,
    register_line_cell_magic,
)


@register_line_magic
def ex1_my_line_magic(line: str) -> str:
    return line


@register_cell_magic
def ex1_my_cell_magic(line: str, cell: str) -> Tuple[str, str]:
    return line, cell


@register_line_cell_magic
def ex1_my_combo_magic(line: str, cell: str = None) -> Union[str, Tuple[str, str]]:
    if cell is None:
        print("Called as line magic")
        return line
    else:
        print("Called as cell magic")
        return line, cell

In [ ]:
# Ex. 1a
foo = %ex1_my_line_magic "hello, infosec jupyterthon!"
print(foo)

In [ ]:
%%ex1_my_cell_magic --args-go-here
foo

In [ ]:
# Ex. 1c
%ex1_my_combo_magic foo!

In [ ]:
%%ex1_my_combo_magic
foo!


---

### Example 2 - Define Custom Magics via Subclass

If you need a custom magic to be stateful, you can define your own magics subclass.

You will decorate your class methods to indicate if it is a line magic, cell magic, or both.

There is one extra step here: you need to register the custom magics class with the IPython global singleton.

Any modules that define a function named `load_ipython_extension` can be imported via `%load_ext`.

In [ ]:
%%writefile magic_module.py
# Ex. 2

from IPython.core.magic import line_magic, cell_magic, line_cell_magic, Magics, magics_class

@magics_class
class ExampleMagic(Magics):
    def __init__(self, shell):

        # Stateful things can be be initialized here.
        # For example: connect to a database, prompt 
        # for authentication, etc.

        super().__init__(shell)
    
    @line_magic
    def ex2_my_class_line_magic(self, line: str) -> str:

        # The `self.shell.user_ns` instance attribute
        # provides magics with access to the notebook's
        # namespace. This means we can reference global
        # variables and even set them!

        # If you want to call a magic from within a
        # function and have access to the local scope,
        # you can wrap the magic with the `@needs_local_scope` 
        # decorator. This will pass in the local scope as the
        # `local_ns` keyword argument.

        # See https://ipython.readthedocs.io/en/stable/config/custommagics.html#accessing-user-namespace-and-local-scope

        user_namespace = self.shell.user_ns

        if line in user_namespace:
            value = user_namespace[line]

            return f"Variable {line} has the assigned value of '{value}'"
        else:
            
            return f"Variable {line} cannot be found in the user namespace!"

    @cell_magic
    def ex2_my_class_cell_magic(self, line: str, cell: str) -> str:
        return f"The provided line was '{line}' and the provided cell contained '{cell}'"

    @line_cell_magic
    def ex2_my_class_combo_magic(self, line: str, cell: str = None) -> str:
        if cell:
            return f"I was executed as a cell magic: {cell}"
        else:
            return "I was executed as a line magic"


def load_ipython_extension(ipython):
    ipython.register_magics(ExampleMagic)

In [ ]:
# Ex. 2a
%reload_ext magic_module

In [ ]:
# Ex. 2b
foo = "bar"
%ex2_my_class_line_magic foo

In [ ]:
%%ex2_my_class_cell_magic example_3_c
cell goes here


---

### Example 3 - Argument Parsing

As mentioned, magics typically parse the line as a commandline tool.

IPython ships with some convenience functions that wrap `argparse` in the `IPython.core.magic_arguments` module.

In [ ]:
# Ex. 3
# From https://ipython.readthedocs.io/en/stable/api/generated/IPython.core.magic_arguments.html
# from IPython.core.magic import register_cell_magic

from IPython.core.magic import register_cell_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring


@magic_arguments()
@argument(
    "--option",
    "-o",
    help=("Add an option here"),
)
@argument(
    "--style",
    "-s",
    default="foo",
    help=("Add some style arguments"),
)
@register_cell_magic
def ex3_my_cell_magic(line, cell):
    args = parse_argstring(ex3_my_cell_magic, line)
    print(f"{args.option=}")
    print(f"{args.style=}")
    print(f"{cell=}")

In [ ]:
%%ex3_my_cell_magic --option foo --style bar
cell goes here

But you can leverage any argument parsing library, such as [`typer`](https://typer.tiangolo.com/) or [`knack`](https://github.com/microsoft/knack). More on this topic later.

---

## When to Use Magics

_Magics are an anti-pattern._

IPython magics are the Python equivalent of necromancy in D&D or the law of equivalent exchange in FMA.

From PEP 20, the [Zen of Python](https://peps.python.org/pep-0020/) for IPython Magics:

| Pros | Cons |
| ---- | ---- |
| Beautiful is better than ugly. | Explicit is better than implicit. |
| Readability counts. | Simple is better than complex. |
| Although practicality beats purity. | Special cases aren't special enough to break the rules. |
| | There should be one-- and preferably only one --obvious way to do it. |
| | If the implementation is hard to explain, it's a bad idea. | 

<br>

---

## Magics for Infosec

_Although practicality beats purity._

| Use Case                     | Example Magics                     |
| ---------------------------- | ---------------------------------- |
| Exploratory data analysis    | `%sql`, `%sparkmagic`, `%kqlmagic` |
| Interop with native tools    | `!`, `%%bash`, `%pwsh`, etc.       |
| Report generation            | `%%jinja`                          |
| Environment management       | `%dotenv`, `%env`                  |
| Workflow or process-oriented | `%jira`, `%bug`                    |
| Text manipulation            | `%iocmagic`, `%base64unpack`       |


---

### Example 4 - Exploratory Data Analysis with `ipython-sql` and `ROADtools`

We can use popular enumeration tools such as `ROADtools` to dump data about an Azure AD tenant.

`ROADtools`, like many other projects, use `sqlite` to persist information.

We can then use the `ipython-sql` magic to interact with these databases from our notebook.

From a terminal, run:

```bash
roadrecon auth --device-code
roadrecon gather
```

This will create a file in the current working directory called `roadrecon.db`.

In [ ]:
# Ex. 4a
%load_ext sql
%sql sqlite:///roadrecon.db

Now we can conveniently query the `sqlite` database using SQL from the comfort of our notebook.

In [ ]:
%%sql

SELECT name FROM sqlite_schema
WHERE type='table'
ORDER BY name;

Better yet, `ipython-sql` has native `pandas` integration so that query results are returned as `pd.DataFrame`.

In [ ]:
# Ex. 4c
results = %sql SELECT * FROM ServicePrincipals;
results_df = results.DataFrame()
results_df.head()

The information above is helpful to identify possibly compromised service principals.

We can use either `pandas` or `SQL` to find service principals with credentials added.

In [ ]:
# Ex. 4d
sps_with_credentials = results_df[
    (results_df.passwordCredentials != "[]") |
    (results_df.keyCredentials != "[]")
]

sps_with_credentials


---

### Example 5 - Report Automation with `jinja2`


We can combine everything we learned above to automate infosec reports in notebooks.

This custom magic passes variables from the notebook namespace into a `jinja2` template inside the cell.

We can rapidly iterate on a `jinja2` template that renders markdown text.

We can then leverage IPython's native `display_markdown` function to display it in the notebook.

In [ ]:
# Ex. 5
from IPython.core.magic import cell_magic, Magics, magics_class
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring
from IPython.display import display_markdown
from jinja2 import Template


@magics_class
class JinjaMagic(Magics):
        
    @magic_arguments()
    @argument(
        "--template-vars",
        help="Variables to render in Jinja2 template",
        nargs="+",
    )
    @cell_magic
    def jinja(self, line: str, cell: str) -> None:

        args = parse_argstring(JinjaMagic.jinja, line)

        template_variables = {
            variable: self.shell.user_ns.get(variable) 
            for variable in args.template_vars
        }

        rendered_template = Template(cell).render(**template_variables)
        display_markdown(rendered_template, raw=True)


ip = get_ipython()
ip.register_magics(JinjaMagic)

Here is an toy example of a mini report that highlights our potentially compromised service principals.

In [ ]:
%%jinja --template-vars sps_with_credentials

### Service Principals with Credentials

**Response Plan**

Please validate that these service principals credentials are legitimate.

Unauthorized credentials can be removed using the [`az ad sp credential delete`](https://learn.microsoft.com/en-us/cli/azure/ad/sp/credential?view=azure-cli-latest#az-ad-sp-credential-delete) command.

{% for sp in sps_with_credentials.to_dict(orient="records") %}

 
| Display Name         | Type                          | Object ID         | App ID         |
| -------------------- | ----------------------------- |------------------ | -------------- |
| {{ sp.displayName }} | {{ sp.servicePrincipalType }} | {{ sp.objectId }} | {{ sp.appId }} |

{% if sp.passwordCredentials != "[]" %}

_Password Credentials Found_:

```json
{{ sp.passwordCredentials }}
```

{% endif %}

{% if sp.keyCredentials != "[]" %}

_Certificate Credentials Found_:

```json
{{ sp.keyCredentials }}
```

{% endif %}

{% endfor %}


---

### Exercise 6 - Magic Wrapper for the Azure CLI `azmagic`

I hope that I have demonstrated how it is easy to use magic commands and even write your own.

Here is an example of one such custom magic that I use during investigations and research: `azmagic`

The Azure CLI (`azcli`) is a very powerful tool. 

It is built from the `azure-sdk-for-python`, but mostly used as a commandline utility.

It can return structured JSON, but wouldn't it be nice if it returned DataFrames?

Microsoft open sourced an argument parsing/CLI framework called `knack` to support `azcli`.

This means the rather messy task of parsing all those args are handled by `knack`, rather than needing to implement them in the magic.

In [ ]:
# Ex. 6
import shlex
import os

import pandas as pd

from typing import Any, Union

from IPython.core.magic import register_line_magic

from azure.cli.core import get_default_cli


@register_line_magic
def az(line: str) -> Union[Any, pd.DataFrame, None]:
    """Implementation of `%az` line magic

    Parameters
    ----------
    line : str
        Arguments to the line magic

    Returns
    -------
    Union[Any, pd.DataFrame, None]
        If the result from the `azcli` command is a List[Dict], then
        this function will attempt to convert them into a `pd.DataFrame`.

        If the `azcli` command raises a SystemExit exception with an error
        code of 0 (a "successful error"), such as when passing the `--help`
        flag, then this function returns `None`.

        Otherwise, this function returns whatever object is the result
        of the `azcli` command.

    Raises
    ------
    SystemExit
        Indicates the `azcli` command was not successful.
    """

    args = shlex.split(line)
    az_cli = get_default_cli()
    out_file = open(os.devnull, "w")

    try:
        exit_code = az_cli.invoke(args, out_file=out_file)

        if exit_code == 0:
            result = az_cli.result.result

            if isinstance(result, list):
                # If we get back a list of dicts,
                # then return a pandas DataFrame
                return pd.json_normalize(result)

            elif isinstance(result, dict):
                # If we get back an OData response,
                # attempt to return the `value` array
                # as a pandas DataFrame.
                if "@odata.context" in result and "value" in result:
                    return pd.json_normalize(result["value"])
                else:
                    return result
            else:
                return result

    except SystemExit as exc:

        # Using the --help flag will raise
        # a SystemExit, but with a successful
        # error code. We don't want to show
        # this exception to the user in the
        # notebook.

        if az_cli.result.error.code == 0:

            return
        else:
            raise exc


In [ ]:
# Ex. 6a
%az

To use it, we first need to login.

In [ ]:
#Ex. 6b
%az login --use-device-code

We can replicate what we collected with `ROADTools`.

In [ ]:
#Ex. 6c
service_principals_df = %az ad sp list --all
service_principals_df.head()

Here is yet another special syntax:

Any variables prefixed with a `$` will be expanded by the line magic. This can be disabled.

In [ ]:
#Ex. 6d
for appId in service_principals_df.head().appId.unique():
    # Note the $appId variable
    app = %az ad sp show --id $appId
    display_name = app["appDisplayName"]
    print(f"{display_name} ({appId})")

But unlike `ROADtools`, which only uses the legacy Azure AD graph to enumerate tenant information, the `azcli` can interact with a wide variety of Azure cloud resources.

In [ ]:
#Ex. 6e
azure_providers = %az provider list
azure_providers

This is very handy for evidence collection purposes.

Here is an example of reading the Azure activity log from the notebook.

In [ ]:
#Ex. 6f
activity_log_last_hour = %az monitor activity-log list --offset 1h
activity_log_last_hour

Even though the `azcli` doesn't officially support the Microsoft Graph beta Reports API, the `az rest` utility will happily obtain the appropriate access tokens and complete your request.

In [ ]:
#Ex. 6g
directory_audit_logs = %az rest --uri https://graph.microsoft.com/beta/auditLogs/directoryAudits --uri-parameters '{"$filter": "activityDateTime ge 2022-11-03"}'
directory_audit_logs

Similarly, the same approach can be used to read the sign-ins reports.


---

## Wrap-Up

- Magic commands are syntactic sugar that make working in the notebook more convenient.
- Custom magic commands are easy to write and can streamline infosec analysis tasks.

Thank you!